# Kelvin-Helmholtz Instability test

Here, we will perform the Kelvin-Helmholtz Instability test with AsterX, and visualize the data saved in OpenPMD format. 

### Background
Kelvin-Helmholtz instability (KHI) is an instability which can develop across fluid interfaces in the presence of a tangential shear flow, resulting in the production of fluid turbulences (imagine vortex-like structures).  In numerical simulations of certain astrophysical scenarios, it is crucial to accurately capture such instability effects as they can have critical implications on the dynamics of the system. For instance, KHI driven turbulences (of length scales ranging from few cm to hundreds of meters) generated in binary neutron star mergers play a significant role in the amplification of magnetic fields.

### Test setup
For the numerical test, KHI simulations can be based on a temporal or a spatial approach. In the temporal case, the fluid flow is considered in a box with periodic (cyclic) boundary conditions,  "moving" with an average speed (absolute instability). In the latter case, simulations mimic a lab experiment with natural inlet and outlet conditions (convective instability).

- This test, performed in 2D, adopts the setup from Springel 2010 (see, for instance, Section 8.7 of https://arxiv.org/abs/0901.4107 for further details).


- Initial values of density $\rho$, 3-velocities ($v^x, v^y, v^z$) and pressure $P$ are set using the thorn 
```KHInitial``` :
    - Grid domain is $[0,1]^2$ with periodic conditions along x. 
    - $\rho = 2.0$, $v^x=+0.5$ within the strip $|y − 0.5| < 0.25$.
    - $\rho = 1.0$, $v^x=-0.5$, elsewhere. $v^z=0$ everywhere.
    - Gas pressure $P=2.5$ everywhere with $\Gamma=5/3$. 
    
 This gives two contact discontinuities or *slip surfaces*.


- To induce the instability, we excite a single mode with wavelength equal to half the domain size by perturbing $v^y$, given by the expression:

 $v^y = w_0 * sin(4 \pi x) * \Big(\exp\big(\frac{-(y - 0.25)^2} {2 \sigma^2}\big) + \exp \big(\frac{-(y + 0.25)^2}{ 2 \sigma^2}\big)\Big)$
  
  where we set $w_0=0.1$, and $\sigma=0.05/\sqrt{2}$.
                    


## 1. Steps to perform the simulation

The parameter file used for this simulation is located within the par directory of the ```KHInitial``` thorn. Let's look at its content first:
<blockquote>

```
ActiveThorns = "
    ADIOS2
    ADMBase
    AsterX
    CarpetX
    HydroBase
    IOUtil
    KHInitial
    ODESolvers
    SystemTopology
    TimerReport
    TmunuBase
    openPMD
"
$nlevels = 1
$ncells = 256

CarpetX::verbose = no

Cactus::presync_mode = "mixed-error"
CarpetX::poison_undefined_values = no

CarpetX::xmin = -0.5
CarpetX::ymin = -0.5
CarpetX::zmin = -0.5 * 2 / $ncells

CarpetX::xmax = 0.5
CarpetX::ymax = 0.5
CarpetX::zmax = 0.5 * 2 / $ncells

CarpetX::ncells_x = $ncells
CarpetX::ncells_y = $ncells
CarpetX::ncells_z = 2
CarpetX::blocking_factor_z = 2

CarpetX::periodic_x = yes
CarpetX::von_neumann_y = yes
CarpetX::von_neumann_z = yes
CarpetX::von_neumann_upper_y = yes
CarpetX::von_neumann_upper_z = yes

CarpetX::max_num_levels = $nlevels
CarpetX::regrid_every = 10
CarpetX::regrid_error_threshold = 0.01

CarpetX::prolongation_type = "ddf"
CarpetX::ghost_size = 2
CarpetX::dtfac = 0.5

ADMBase::set_adm_variables_during_evolution = yes
ADMBase::initial_data = "Cartesian Minkowski"
ADMBase::initial_lapse = "one"
ADMBase::initial_shift = "zero"
ADMBase::initial_dtlapse = "none"
ADMBase::initial_dtshift = "none"

HydroBase::initial_hydro = "KHI"
KHInitial::gamma = 1.6667
KHInitial::w0 = 0.1
KHInitial::sigma = 0.0353553390593274   # 0.05/sqrt(2)
KHInitial::rhoUp = 1.0
KHInitial::vxUp = -0.5
KHInitial::rhoLow = 2.0
KHInitial::vxLow = 0.5
KHInitial::p_val= 2.5

AsterX::gamma = 1.6667
AsterX::reconstruction_method = "minmod"
AsterX::max_iter = 100

Cactus::terminate = "time"
Cactus::cctk_final_time = 2.0
ODESolvers::method = "RK4"

IO::out_dir = $parfile
IO::out_every = 50
CarpetX::out_silo_vars = "
    HydroBase::rho
    HydroBase::vel
    HydroBase::eps
    HydroBase::press
    CarpetX::regrid_error
"

CarpetX::out_openpmd_vars = "
    HydroBase::rho
    HydroBase::vel
    HydroBase::eps
    HydroBase::press
    CarpetX::regrid_error
"

TimerReport::out_every = 100
TimerReport::out_filename = "TimerReport"
TimerReport::output_all_timers_together = yes
TimerReport::output_all_timers_readable = yes
TimerReport::n_top_timers = 50
```

Now, let's first move to the Cactus folder

In [ ]:
cd ~/Cactus

Then, submit the simulation using the following command:

In [ ]:
!./simfactory/bin/sim submit KHI --parfile=./arrangements/CarpetX/KHInitial/par/KHI.par --config=sim-gpu --procs=1 --walltime=00:20:00

The above command creates and runs the simulation ```KHI```, using the configuration ```sim-gpu```. The data is saved in the directory ```./simulations/KHI```.



## 2. Steps to visualize simulation data

The 2D data is saved in both Silo format (which can be visualised, for instance, via VisIt) and in OpenPMD format. 

For further info on Silo, please visit: https://wci.llnl.gov/simulation/computer-codes/silo

For further info about OpenPMD, please visit:

- Official website:  https://www.openpmd.org
- GitHub repository: https://github.com/openPMD
- Documentation:     https://openpmd-api.readthedocs.io

Now, let's go back to the home directory:

In [ ]:
cd ~/

Import all the required modules:

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from celluloid import Camera
import openpmd_api as io

Set the Matplotlib backend to `notebook`, not `inline`, since we'll want to animate some figures and the latter is not compatible with that

In [ ]:
%matplotlib notebook

Open a .bp file (ADIOS2 extension) as an OpenPMD **'series'**, which is a collection of **'iterations'**, each of which contains **'records'**, which are sets of either structured data --- **'meshes'** --- or unstructured data --- **'particles'**; AsterX only outputs mesh data. Each record has one or more **'components'**: for example, a record representing a scalar field only has one component, while a record representing a vector field has three.

In [ ]:
series = io.Series("./simulations/KHI/output-0000/KHI/KHI.it%08T.bp", io.Access.read_only)

All iterations in our series have the same structure --- i.e., they contain
the same records --- since they all represent the same output, just at
different times. Here we define an empty Python nested dictionary whose
structure, once full, will be:

Iteration 0:
- Record 1:
  - Component 1: 3D data array
  - Component 2: 3D data array
  - Component 3: 3D data array
- Record 2:
  - Component 1: 3D data array
  - Component 2: 3D data array
  - Component 3: 3D data array
  
 [...]

Iteration 1:
- Record 1:
  - Component 1: 3D data array
  - Component 2: 3D data array
  - Component 3: 3D data array
- Record 2:
  - Component 1: 3D data array
  - Component 2: 3D data array
  - Component 3: 3D data array
  
 [...]

[...]

In [ ]:
iter_rec_comp_dict = {}

Print info, register data chunks and fill the above dictionary:

In [ ]:
for index in series.iterations:
    iteration = str(index)

    print("\nIteration " + iteration + ":")
    print("==============")

    # Allocate an empty dictionary associated to this iteration
    iter_rec_comp_dict[iteration] = {}

    i = series.iterations[index]

    for key in i.meshes:
        print("Components of record \"" + key + "\":")

        # Allocate an empty dictionary associated to this record. Notice that
        # 'record' is an OpenPMD mesh object, so it's better to use 'key'
        # instead of 'record' as a key in the dictionary ('record' could also be
        # used, but it makes accessing the key clumsy).
        record = i.meshes[key]
        iter_rec_comp_dict[iteration][key] = {}

        # Load each component of each record as a 'data chunk', i.e., an
        # allocated, but STILL INVALID, NumPy array. Later we will flush all
        # chunks (i.e., basically, fill the NumPy arrays) at once: this leads
        # to better I/O performance compared to flushing a large number of
        # small chunks. That's why we bothered creating the nested dictionary:
        # in this way, we can access the valid NumPy arrays for plotting
        # without having to flush each single chunk.
        # *IMPORTANT*: DO NOT access data chunks until flushing has happened!
        for component in record:
            print("    > " + component)  # 'component' is a string
            iter_rec_comp_dict[iteration][key][component] = record[component].load_chunk()  # *INVALID* 3D NumPy array

        print("")

Flush all registered data chunks, which are now **VALID** 3D NumPy arrays:

In [ ]:
series.flush()

Visualize a 2D movie of the mass density on an (x, z) slice of the domain

In [ ]:
# Select the desired record and component to plot
record    = "hydrobase_rho_rl00"  # "carpetx_regrid_error_rl00", "hydrobase_eps_rl00", "hydrobase_press_rl00", "hydrobase_rho_rl00", "hydrobase_vel_rl00"
component = "hydrobase_rho"  # "carpetx_regrid_error", "hydrobase_eps", "hydrobase_press", "hydrobase_rho", "hydrobase_velx", "hydrobase_vely", "hydrobase_velz"

# Set up the axes for the plot and the colorbar
fig    = plt.figure(figsize = [9., 4.5])
axplot = fig.add_axes([0.07, 0.14, 0.82, 0.76])
axclb  = fig.add_axes([0.92, 0.14, 0.02, 0.75])

# Set title and labels
axplot.set_title(component, fontsize = 18., fontweight = "bold", color = "midnightblue")
axplot.set_xlabel("x [arb. units]", fontsize = 15.)
axplot.set_ylabel("z [arb. units]", fontsize = 15.)

# Initialize the camera
camera = Camera(fig)

# Print frames
for iteration in iter_rec_comp_dict:
    # Retrieve the 3D array containing the data
    array3D = iter_rec_comp_dict[iteration][record][component]
    
    # Plot on the (x, y) plane at the half-way value of z
    # (z index = int(array3D.shape[1]/2) )
    # Notice that the 3D array is stored in (z, y, x) order
    z_index = int(array3D.shape[0]/2)
    image   = axplot.pcolormesh(array3D[z_index, :, :],
                                cmap = "magma", vmin = 0.4, vmax = 2.5)
                                ##vmin = np.min(data_xz), vmax = np.max(data_xz))

    # Set up the colorbar
    fig.colorbar(image, cax = axclb, extend = "neither")
    
    # Print the current iteration
    #fig.text(0.65, 0.72, "Iteration " + iteration,
    #         fontsize = 15., fontweight = "bold", color = "white")

    # Take a snapshot of the figure at this iteration (needed later for the animation)
    camera.snap()

In [ ]:
animation = camera.animate()
plt.show()